# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## JuMP: Julia for Mathematical Programming

### Problem 2: The Transportation, Transhipment and Assignment problem

* This example was adapted from ["Arenales, M., Morabito, R., Armentano, V., & Yanasse, H. (2015). Pesquisa operacional: para cursos de engenharia. Elsevier Brasil."](https://www.amazon.com.br/Pesquisa-Operacional-Para-cursos-engenharia-ebook/dp/B016APOY5U "Arenales et al. (2015): \"PO para Engenharia\"")

A beverage distributor has two production centers, Reggio Emilia and Bologna, two warehouses, Parma and Firenze, and three consumer markets, Roma, Napoli and Torino. The production centers must send the products to the warehouses and then transport them to the consumer markets. Deposits have no storage capacity. In the sequence, we show the data of this problem. Note: **the data are artificially chosen. They do not represent the reality**.

In [1]:
# OR libraries
using JuMP
using Clp

# Production centers
S = ["Reggio Emilia","Bologna"]

# Warehouses (depots)
D = ["Parma","Firenze"]

# Consumer markets
C = ["Roma","Napoli","Torino"]

# Demands
d = Dict(
    "Roma" => 500,
    "Napoli" => 400,
    "Torino" => 900
)

# Supply
o = Dict(
    "Reggio Emilia" => 800,
    "Bologna" => 1000
)

# Transportation costs
c = Dict(
    ("Reggio Emilia","Parma") => 1,
    ("Reggio Emilia","Firenze") => 3,
    ("Bologna","Parma") => 1,
    ("Bologna","Firenze") => 2,
    ("Parma","Roma") => 1,
    ("Parma","Napoli") => 3,
    ("Parma","Torino") => 3,
    ("Firenze","Roma") => 3,
    ("Firenze","Napoli") => 4,
    ("Firenze","Torino") => 1
)

Dict{Tuple{String,String},Int64} with 10 entries:
  ("Bologna", "Firenze")       => 2
  ("Reggio Emilia", "Parma")   => 1
  ("Firenze", "Napoli")        => 4
  ("Firenze", "Torino")        => 1
  ("Bologna", "Parma")         => 1
  ("Parma", "Roma")            => 1
  ("Firenze", "Roma")          => 3
  ("Parma", "Napoli")          => 3
  ("Parma", "Torino")          => 3
  ("Reggio Emilia", "Firenze") => 3

We perform transportation from production centers to depots, and from depots to consumer markets. Then, all combinations we have are:

In [2]:
# Library that will do the combinations of transportation we have among cities
using IterTools

A = Iterators.product(D,C)
B = Iterators.product(S,D)
A = union(A,B)

10-element Array{Tuple{String,String},1}:
 ("Parma", "Roma")           
 ("Firenze", "Roma")         
 ("Parma", "Napoli")         
 ("Firenze", "Napoli")       
 ("Parma", "Torino")         
 ("Firenze", "Torino")       
 ("Reggio Emilia", "Parma")  
 ("Bologna", "Parma")        
 ("Reggio Emilia", "Firenze")
 ("Bologna", "Firenze")      

**Objective**: to meet consumer market demand, minimizing costs.

#### Mathematical modeling

* **Libraries**

In [3]:
using JuMP, Clp

* **Building a model object**

In [4]:
model = Model(with_optimizer(Clp.Optimizer)) 

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.

* **Variables**
    * $x_{ij} \ge 0$: quantity of beverages transported from $i$ to $j$.

In [5]:
@variable(model, x[A] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, Tuple{String,String}[("Parma", "Roma"), ("Firenze", "Roma"), ("Parma", "Napoli"), ("Firenze", "Napoli"), ("Parma", "Torino"), ("Firenze", "Torino"), ("Reggio Emilia", "Parma"), ("Bologna", "Parma"), ("Reggio Emilia", "Firenze"), ("Bologna", "Firenze")]
And data, a 10-element Array{VariableRef,1}:
 x[("Parma", "Roma")]           
 x[("Firenze", "Roma")]         
 x[("Parma", "Napoli")]         
 x[("Firenze", "Napoli")]       
 x[("Parma", "Torino")]         
 x[("Firenze", "Torino")]       
 x[("Reggio Emilia", "Parma")]  
 x[("Bologna", "Parma")]        
 x[("Reggio Emilia", "Firenze")]
 x[("Bologna", "Firenze")]      

* **Objective function**: minimize transportation costs.

\begin{equation}
\min f(x) = \sum_{(i,j) \in A} c_{ij}x_{ij}
\end{equation}

In [6]:
@objective(model, Min, sum(c[a]*x[a] for a in A))

x[("Parma", "Roma")] + 3 x[("Firenze", "Roma")] + 3 x[("Parma", "Napoli")] + 4 x[("Firenze", "Napoli")] + 3 x[("Parma", "Torino")] + x[("Firenze", "Torino")] + x[("Reggio Emilia", "Parma")] + x[("Bologna", "Parma")] + 3 x[("Reggio Emilia", "Firenze")] + 2 x[("Bologna", "Firenze")]

* **Constraints**: 

    * Quantity transported from sources must respect their supplies.
    \begin{equation}
    \sum_{j \in D} x_{ij} \le o_i, \forall i \in S
    \end{equation}

    * Quantity transported to destination must meet the demands
    \begin{equation}
    \sum_{i \in D} x_{ij} = d_j, \forall j \in C
    \end{equation}

    * Flow control of intermediate nodes
    \begin{equation}
    \sum_{i \in S} x_{ij} = \sum_{k \in C} x_{jk}, \forall j \in D
    \end{equation}

In [7]:
@constraints(model, begin
        cstrS[i in S], sum(x[(i,j)] for j in D) <= o[i]
        cstrC[j in C], sum(x[(i,j)] for i in D) == d[j]
        cstrD[j in D], sum(x[(i,j)] for i in S) == sum(x[(j,k)] for k in C)
        end)

* **Optimize it!**

In [8]:
start = time()
optimize!(model)
println("Time spent: $(time()-start)s")

Time spent: 10.881999969482422s
Coin0506I Presolve 4 (-3) rows, 7 (-3) columns and 14 (-6) elements
Clp0006I 0  Obj 4000 Primal inf 1800 (1) Dual inf 2.999998 (2)
Clp0006I 4  Obj 5300
Clp0000I Optimal - objective value 5300
Coin0511I After Postsolve, objective 5300, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 5300 - 4 iterations time 0.042, Presolve 0.04


* **Printing the solution**

In [9]:
println("Objective function = ", objective_value(model))
for i in A
    if(value(x[i]) > 0.0)
        println("x[", i, "]: ", value(x[i]))
    end
end

Objective function = 5300.0
x[("Parma", "Roma")]: 500.0
x[("Parma", "Napoli")]: 400.0
x[("Firenze", "Torino")]: 900.0
x[("Reggio Emilia", "Parma")]: 800.0
x[("Bologna", "Parma")]: 100.0
x[("Bologna", "Firenze")]: 900.0


* **Printing dual variables**

In [10]:
println("Dual variables for the first set of constraints: ", [dual(cstrS[i]) for i in S])
println("Dual variables for the second set of constraints: ", [dual(cstrC[i]) for i in C])
println("Dual variables for the third set of constraints: ", [dual(cstrD[i]) for i in D])

Dual variables for the first set of constraints: [0.0, 0.0]
Dual variables for the second set of constraints: [2.0, 4.0, 3.0]
Dual variables for the third set of constraints: [1.0, 2.0]
